In [ ]:
# Google drive authentication (for use with google colab).
from google.convLayeronvLayerolab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Library imports
import numpy as np
import nibabel as nib

import os
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Model, utils
from tensorflow.keras.layers import Conv3D, MaxPool3D, UpSampling3D, concatenate, BatchNormalization,Activation,Concatenate,Conv2DTranspose,Input,Conv3DTranspose,MaxPooling3D,Dropout
from tensorflow.keras.optimizers import Adam
from keras.models import load_model
import segmentation_models_3D as sm

import random 

from tensorflow.keras import backend as K 

2022-06-24 15:55:02.416446: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-24 15:55:02.416492: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Segmentation Models: using `tf.keras` framework.


In [ ]:
# Function to create a filename for the brats 2021 dataset
def nameBuilder(num):
  baseName = "BraTS2021_"
  name = baseName + num.zfill(5)
  return name


In [ ]:
# Function to create lists of file directories to be passed into the data generator

trainList = os.listdir("/content/drive/MyDrive/output/train/image")
valList = os.listdir("/content/drive/MyDrive/output/val/image")
testList = os.listdir("/content/drive/MyDrive/output/test/image")

trainImageList = []
trainMaskList = []

valImageList = []
valMaskList = []

testImageList = []
testMaskList = []

l = len(trainList) 
for i in range(l):
  num = (trainList[i].split("_")[1])
  name = (nameBuilder(num))
  trainImageList.append(os.path.join("/content/drive/MyDrive/output/train/image", name +"_image.npy"))
  trainMaskList.append(os.path.join("/content/drive/MyDrive/output/train/mask", name +"_mask.npy"))


l = len(valList) 
for i in range(l):
  num = (valList[i].split("_")[1])
  name = (nameBuilder(num))
  valImageList.append(os.path.join("/content/drive/MyDrive/output/val/image", name +"_image.npy"))
  valMaskList.append(os.path.join("/content/drive/MyDrive/output/val/mask", name +"_mask.npy"))

l = len(testList) 
for i in range(l):
  num = (testList[i].split("_")[1])
  name = (nameBuilder(num))
  testImageList.append(os.path.join("/content/drive/MyDrive/output/test/image", name +"_image.npy"))
  testMaskList.append(os.path.join("/content/drive/MyDrive/output/test/mask", name +"_mask.npy"))
del(trainList)
del(valList)
del(testList)

920
115


In [ ]:
# Data generator functions

def load(imgList):
    list = []
    for img in imgList:
        img = np.load(img)
        list.append(img)
    arr = np.array(list)
    return arr

def imageGenertor(batchSize, imageList, maskList):

    length = len(imageList)
    while True:
        start = 0
        end = batchSize

        while start < length:
            x = load(imageList[start:min(end, length)])
            y = load(maskList[start:min(end, length)])
            
            yield(x,y)

            start += batchSize
            end += batchSize

In [ ]:
# U-Net model defined using Tensorflow and Keras

def segmentationModel(HEIGHT, WIDTH, DEPTH, CHANNELS, CLASSES):

    input = Input((HEIGHT, WIDTH, DEPTH, CHANNELS))

    kernel_initializer =  'he_uniform'

    convLayer1 = Conv3D(16, (3, 3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(input)
    convLayer1 = Dropout(0.1)(convLayer1)
    convLayer1 = Conv3D(16, (3, 3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(convLayer1)
    pool1 = MaxPooling3D((2, 2, 2))(convLayer1)
    
    convLayer2 = Conv3D(32, (3, 3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(pool1)
    convLayer2 = Dropout(0.1)(convLayer2)
    convLayer2 = Conv3D(32, (3, 3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(convLayer2)
    pool2 = MaxPooling3D((2, 2, 2))(convLayer2)
     
    convLayer3 = Conv3D(64, (3, 3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(pool2)
    convLayer3 = Dropout(0.2)(convLayer3)
    convLayer3 = Conv3D(64, (3, 3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(convLayer3)
    pool3 = MaxPooling3D((2, 2, 2))(convLayer3)
     
    convLayer4 = Conv3D(128, (3, 3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(pool3)
    convLayer4 = Dropout(0.2)(convLayer4)
    convLayer4 = Conv3D(128, (3, 3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(convLayer4)
    pool4 = MaxPooling3D(pool_size=(2, 2, 2))(convLayer4)
     
    convLayer5 = Conv3D(256, (3, 3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(pool4)
    convLayer5 = Dropout(0.3)(convLayer5)
    convLayer5 = Conv3D(256, (3, 3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(convLayer5)
    
    upSample1 = Conv3DTranspose(128, (2, 2, 2), strides=(2, 2, 2), padding='same')(convLayer5)
    upSample1 = concatenate([upSample1, convLayer4])
    convLayer6 = Conv3D(128, (3, 3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(upSample1)
    convLayer6 = Dropout(0.2)(convLayer6)
    convLayer6 = Conv3D(128, (3, 3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(convLayer6)
     
    upSample2 = Conv3DTranspose(64, (2, 2, 2), strides=(2, 2, 2), padding='same')(convLayer6)
    upSample2 = concatenate([upSample2, convLayer3])
    convLayer7 = Conv3D(64, (3, 3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(upSample2)
    convLayer7 = Dropout(0.2)(convLayer7)
    convLayer7 = Conv3D(64, (3, 3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(convLayer7)
     
    upSample3 = Conv3DTranspose(32, (2, 2, 2), strides=(2, 2, 2), padding='same')(convLayer7)
    upSample3 = concatenate([upSample3, convLayer2])
    convLayer8 = Conv3D(32, (3, 3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(upSample3)
    convLayer8 = Dropout(0.1)(convLayer8)
    convLayer8 = Conv3D(32, (3, 3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(convLayer8)
     
    upSample4 = Conv3DTranspose(16, (2, 2, 2), strides=(2, 2, 2), padding='same')(convLayer8)
    upSample4 = concatenate([upSample4, convLayer1])
    convLayer9 = Conv3D(16, (3, 3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(upSample4)
    convLayer9 = Dropout(0.1)(convLayer9)
    convLayer9 = Conv3D(16, (3, 3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(convLayer9)
     
    outputs = Conv3D(CLASSES, (1, 1, 1), activation='softmax')(convLayer9)
     
    model = Model(inputs=[input], outputs=[outputs])    
    return model

In [37]:
image = np.stack([image, image, image, image], axis=3)
# hStart = int((height - 128) / 2)
# hEnd = int((height - ((height-128) / 2)))

# wStart = int((width - 128) / 2)
# wEnd = int((width - ((width-128) / 2)))

# dStart = int((depth - 128) / 2)
# dEnd = int((depth - ((depth-128) / 2)))

# print(hStart, hEnd)
# print(wStart, wEnd)
# print(dStart, dEnd)
# # 56:184, 56:184, 13:141

In [10]:
image = image[hStart:hEnd, wStart:wEnd, dStart:dEnd]
print(image.shape)

(128, 128, 128, 4)


In [ ]:
# Import and define loss function

import segmentation_models_3D as sm
diceLoss = sm.losses.DiceLoss()

In [ ]:
def dice_coef(y_true, y_pred):
    """_summary_
    Dice Coefficienet function for single class data
    Adapted from: https://stackoverflow.com/questions/61488732/how-calculate-the-dice-coefficient-for-multi-class-segmentation-task-using-pytho

    Args:
        y_true (_type_): _description_
        y_pred (_type_): _description_

    Returns:
        _type_: _description_
    """
    y_true_f = y_true.flatten()
    y_pred_f = y_pred.flatten()
    intersection = np.sum(y_true_f * y_pred_f)
    smooth = 0.0001
    return (2. * intersection + smooth) / (np.sum(y_true_f) + np.sum(y_pred_f) + smooth)

def dice_coef_multilabel(y_true, y_pred, numLabels):
    """_summary_
    Dice Coefficienet function for multi class data
    Adapted from: https://stackoverflow.com/questions/61488732/how-calculate-the-dice-coefficient-for-multi-class-segmentation-task-using-pytho

    Args:
        y_true (_type_): _description_
        y_pred (_type_): _description_
        numLabels (_type_): _description_

    Returns:
        _type_: _description_
    """
    if y_true.shape != (128,128,128) or y_pred.shape != (128,128,128):
        return False
    y_true = to_categorical(y_true, num_classes=numLabels)
    y_pred = to_categorical(y_pred, num_classes=numLabels)
    dice=0
    for index in range(numLabels):
        dice += dice_coef(y_true[:,:,:,index], y_pred[:,:,:,index])
    return dice/numLabels 

In [ ]:
# Define model
model = segmentationModel(128, 128, 128, 4, 4)

In [ ]:
# Define batchsize and data generator objects for the training and validation sets

batchSize = 4
datagen = imageGenertor(batchSize, "/content/drive/MyDrive/output/train",trainImageList, trainMaskList)
# img, msk = datagen.__next__()
# print(img.shape)
# print(msk.shape)
valdatagen = imageGenertor(batchSize, "/content/drive/MyDrive/output/val",valImageList, valMaskList)
# vimg, vmsk = valdatagen.__next__()
# print(vimg.shape)

In [ ]:
# Training or fitting the model to a training dataset

steps = len(trainImageList) // batchSize
valsteps =  len(valImageList) // batchSize
epochs = 10
modelSaveName = "segmentationModel" 

# For use when retraining a previsouly trained model
# model = load_model("/content/drive/MyDrive/models/cross15", custom_objects={'loss': tf.keras.losses.CategoricalCrossentropy, 'dice_coefficient':dice_coefficient})

# Compile the model 
model.compile(optimizer='Adam',loss=tf.keras.losses.CategoricalCrossentropy(), metrics=[dice_coef_multilabel])

# Fit the model, saving training metrics in history variable
history = model.fit(x=datagen,steps_per_epoch=steps, validation_data=valdatagen, validation_steps=valsteps, epochs=epochs, verbose=1)

# Save the model
model.save(f"/content/drive/MyDrive/models/{modelSaveName}")


Epoch 1/10
  2/230 [..............................] - ETA: 6:31 - loss: 1.3508 - dice_coefficient: 0.1198   

In [ ]:
model.save(f"/content/drive/MyDrive/models/{modelSaveName}")


NameError: ignored

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

path = "/content/drive/MyDrive/RAWDATA/MICCAI_BraTS2020_ValidationData"
total = (len(os.listdir(path)))
dir = os.listdir(path)
num = 0
for d in dir:
    num += 1
    curPath = os.path.join(path, d)
    # Flair
    flairPath = os.path.join(curPath, d+"_flair.nii")
    flairImage = nib.load(flairPath).get_fdata()
    scaledFlair = scaler.fit_transform(flairImage.reshape(-1, flairImage.shape[-1])).reshape(flairImage.shape)
    # flairImage = scaler.fit_transform(flairImage.reshape(-1, flairImage.shape[-1])).reshape(flairImage.shape)
    
    # T1CE
    t1cePath = os.path.join(curPath, d+"_t1ce.nii")
    t1ceImage = nib.load(t1cePath).get_fdata()
    scaledT1ce = scaler.fit_transform(t1ceImage.reshape(-1, t1ceImage.shape[-1])).reshape(t1ceImage.shape)

    #T1
    t1Path = os.path.join(curPath, d+"_t1.nii")
    t1Image = nib.load(t1Path).get_fdata()
    scaledT1 = scaler.fit_transform(t1Image.reshape(-1, t1Image.shape[-1])).reshape(t1Image.shape)

    # T2
    t2Path = os.path.join(curPath, d+"_t2.nii")
    t2Image = nib.load(t2Path).get_fdata()
    scaledT2 = scaler.fit_transform(t2Image.reshape(-1, t2Image.shape[-1])).reshape(t2Image.shape)

    
   

    # # Mask
    # maskPath = os.path.join(curPath, d + "_seg.nii.gz")
    # maskImage = nib.load(maskPath).get_fdata()
    # maskImage = maskImage.astype(np.uint8)
    # maskImage[maskImage==4] = 3

    # Combine Modalities

    combinedImage = np.stack([scaledFlair, scaledT1ce, scaledT1, scaledT2], axis=3)


    # Crop volumes to 128x128x128
    combinedImage = combinedImage[56:184, 56:184, 13:141]
    # maskImage = maskImage[56:184, 56:184, 13:141]


    print(f"Saved: {num} out of {total}")
    # Save as numpy arrays
    imageSavePath = "/content/drive/MyDrive/VAL/image"
    imageSaveName = d + "_image"
    print(imageSaveName)
    np.save(os.path.join(imageSavePath, imageSaveName), combinedImage)
    
    
    



    values, counts = np.unique(maskImage, return_counts=True)

    if (1 - (counts[0]/counts.sum())) > 0.01:
        print(f"Saved: {num} out of {total}")
         # Save as numpy arrays
        imageSavePath = "/content/drive/MyDrive/data2/image"
        imageSaveName = d + "_image"
        np.save(os.path.join(imageSavePath, imageSaveName), combinedImage)

        maskSavePath = "/content/drive/MyDrive/data2/mask"
        maskSaveName = d + "_mask"
        maskImage = to_categorical(maskImage, num_classes=4)#.astype(np.uint8)
        np.save(os.path.join(maskSavePath, maskSaveName), maskImage)
        
     
        
   
  
    